# Prob 2 Part 2 - exam

In [26]:
import numpy as np
import scipy as sp
import math

## Question 9

In [27]:
la = 20
mu = 7.5

S = np.array([
    [-la, la,0,  0, 0],
    [mu, -la-mu,la,  0, 0],
    [0,2*mu, -la-2*mu, la, 0,],
    [0,0, 3*mu, -la-3*mu, la],
    [0,0,0,4*mu, -la-4*mu]
])
a = np.array([1,0,0,0,0])
def exp_matrix(A,t):
    return sp.linalg.expm(A*t)

expSt = exp_matrix(S,4)
a@expSt@np.ones(S.shape[0])
prob = 1-a@expSt@np.ones(S.shape[0])
prob

0.9991375651028974

### Question 10

In [28]:
la = 20
mu = 7.5
s = 4
pi0_inv = 0
for j in range(s):
    pi0_inv += 1/math.factorial(j)*(la/mu)**j #formula on page 458 of Pinsky & Karlin
pi0_inv += (la/mu)**s/(math.factorial(s)*(1-la/(s*mu)))
pi0 = 1/pi0_inv
L0 = pi0/s*(la/mu)**s*(la/(s*mu))/(1-la/(s*mu))**2

print(L0), (pi0)

4.541019955654101


(None, 0.059866962305986704)

### Question 11

In [29]:
def pi_k(k,la,mu,s,pi0): #page 458 of Pinsky & Karlin
    if k < s:
        return pi0*(la/mu)**k/math.factorial(k)
    else:
        return 1/math.factorial(s)*pi0*(la/mu)**s*(la/(s*mu))**(k-s)

In [30]:
import math
import scipy.stats as st

def erlang_c(lam, mu, s):
    a = lam/mu
    rho = lam/(s*mu)
    num = a**s / (math.factorial(s) * (1-rho))
    den = sum(a**k / math.factorial(k) for k in range(s)) + num
    return num/den

lam = 20.0
mu  = 7.5
s   = 4
t   = 10/60  # 10 minutes in hours

C = erlang_c(lam, mu, s)
P_tail_closed = C * math.exp(-(s*mu - lam)*t)
print("closed form:", P_tail_closed)

closed form: 0.07147399013513056


# Question 12

In [ ]:
def pi_n(n,la=la,mu=mu,s=s,pi0=pi0):
    return pi_k(n,la,mu,s,pi0)

In [ ]:

#Parameters for (M/M/s) system
lam = 20.0
mu  = 7.5
s   = 4
rho = lam*1/mu #trafic intensity first system

#Calculate rate of exit from (M/M/s) system
rate_exit = 0
pi_aggregate = 0
for i in range(s):
    pi_aggregate += pi_n(i)
    rate_exit += pi_n(i)*mu*i
C = 1-(pi_aggregate)
rate_exit += C*s*mu

#Calculate expected time in (M/M/s) system
W_cc = 1/(mu*(s-rho))*C
print("W_cc:", W_cc)
E_cc = 1/mu+W_cc
print("W_cc:", W_cc)
print("E_cc:", E_cc)

#Calculate entry rate 
rate_enter = rate_exit*0.05
print("rate enter:", rate_enter)

#Parameters for (M/G/1) system
la_cf = rate_enter
v_cf = 1/2 #mean -service distribution
tau_cf = 1/2 #sd - service distribution
rho_cf = la_cf*v_cf #trafic intensity

print("rho_cf:", rho_cf)

w = (la_cf*(tau_cf**2+v_cf**2))/(2*(1-rho_cf)) # waiting time
E_cf = w + v_cf
print("E_cf:", E_cf)
print("waiting time:", w)
#Total expected time in the system for customer that needs service in both systems
E_total = E_cc + E_cf
print("E_total:", E_total)

W_cc: 0.037841832963784176
W_cc: 0.037841832963784176
E_cc: 0.17117516629711751
rate enter: 1.0
rho_cf: 0.5
E_cf: 1.0
waiting time: 0.5
E_total: 1.1711751662971175
